###Clustering Businesses

In [38]:
import os
import csv
import re

business_csv_file = open(os.path.join("..","data","biz_csv","business.csv"),"r")
business_csv = csv.reader(business_csv_file)

categories = list()
for line in business_csv:
    categories.append(line[-1])
categories = categories[1:]

business_csv_file.close()

unique_cats = set()
for category in categories:
    cat_list = re.split(",|&",category)
    for cat in cat_list:
        cat = cat.lower().strip()
        if len(cat) > 0:
            unique_cats.add(cat.lower().strip())

print("There are "+str(len(unique_cats))+" unique categories and subcategories for all businesses")

There are 1310 unique categories and subcategories for all businesses


In [41]:
import os
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import csv

business_csv_file = open(os.path.join("..","data","biz_csv","business.csv"),"r")
business_csv = csv.reader(business_csv_file)

categories = list()
for line in business_csv:
    categories.append(line[-1])
categories = categories[1:]

class CustomTdidfVectorizer(TfidfVectorizer):
    def build_tokenizer(self):
        token_pattern = re.compile(",")
        return lambda doc: token_pattern.findall(doc)
#         tokenize = super(TfidfVectorizer, self).build_tokenizer()
#         return lambda doc: list(to_british(tokenize(doc)))

vectorizer = TfidfVectorizer(stop_words='english') # term-frequency x inverse-document frequency
# tokenize based on comma instead of space. Otherwise, words like "Public Services" will not be seen as a single word 
# by the vectorizer
cat = vectorizer.fit_transform(categories)

no_of_clusters = 20

kmeans_model = KMeans(n_clusters=20, init='k-means++', max_iter=100, n_init=1,random_state=1)
kmeans_model.fit(cat)

business_csv_file.close()

business_csv_file = open(os.path.join("..","data","biz_csv","business.csv"),"r")
business_csv_clustered_file = open(os.path.join("..","data","biz_csv","business_clustered.csv"),"w+")
business_csv = csv.reader(business_csv_file)
business_clustered_csv = csv.writer(business_csv_clustered_file)

for line in business_csv:
    if not (line[-1] == "categories"):
        line.extend(kmeans_model.predict(vectorizer.transform(line[-1:])))
        business_clustered_csv.writerow(line)

business_csv_clustered_file.close()
business_csv_file.close()

print("Done")

Done
